In [9]:
from numpy import pi, linspace, diff, mean, min, max, arctan2
from skyfield.api import load
from skyfield.framelib import ecliptic_J2000_frame
from skyfield.searchlib import find_discrete
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [10]:
ts = load.timescale()
# eph_name = 'de421.bsp'
eph_name = 'de422.bsp'
t_start = ts.tt(1750, 6, 21, 0, 0, 0)
eph = load(eph_name)

## Orbit angle to the elliptical

In [11]:
def get_inclination_ecliptic(obj, ref, t_start, years):
    def f_vel():
        def zero(t):
            _, vel = (obj.at(t) - ref.at(t)).frame_xyz_and_velocity(ecliptic_J2000_frame)
            return vel.km_per_s[2] > 0
        zero.step_days = 10.0
        return zero
    
    zero_vel = find_discrete(t_start, t_start + 365.25 * years, f_vel())
    t_high = zero_vel[0][zero_vel[1] == 0]
    lat, _, _ = (obj.at(t_high) - ref.at(t_high)).frame_latlon(ecliptic_J2000_frame)
    return t_high, lat

In [12]:
for p in ["mercury", "venus", "moon", "mars", "jupiter_barycenter", "saturn_barycenter", "uranus_barycenter", "neptune_barycenter"]:
    ref = eph["sun"]
    years = 400
    if p.lower() == "moon":
        ref = eph["earth"]
        years = 20
    obj = eph[p]
    t_high, lat = get_inclination_ecliptic(obj, ref, t_start, years)
    colors = [f"#{255:02x}{int(255 * (t_high.tt[-1] - t) / (t_high.tt[-1] - t_high.tt[0])):02x}{255:02x}" for t in t_high.tt]
    fig = figure(title=p.capitalize() + ", orbit angle to ecliptic, starting at " + t_start.tt_strftime(), x_axis_label="years", y_axis_label="degrees", width=1100)
    fig.scatter((t_high.tt - t_high.tt[0])/365.25, lat.degrees , fill_color=colors, line_color="red", size = 5)
    fig.line((t_high.tt - t_high.tt[0])/365.25, lat.degrees, line_color="blue")

    show(fig)